# SDM

In [99]:
using SpeciesDistributionToolkit
using CairoMakie
using Statistics
CairoMakie.activate!(px_per_unit = 6.0)

In [ ]:
tx = taxon("Akodon montensis")

In [ ]:
place = SpeciesDistributionToolkit.gadm("PRY")
extent = SpeciesDistributionToolkit.boundingbox(place)

In [ ]:
provider = RasterData(CHELSA2, BioClim)

In [ ]:
L = SDMLayer{Float16}[mask!(SDMLayer(provider; layer=l, extent...), place) for l in ["BIO1", "BIO12", "BIO3", "BIO8"]]

In [ ]:
presences = occurrences(tx, first(L), "occurrenceStatus" => "PRESENT", "country" => "PY", "limit" => 300, "hasCoordinate" => true)
occurrences!(presences)

In [ ]:
presencelayer = mask(first(L), presences)
background = pseudoabsencemask(DistanceToEvent, presencelayer)
bgpoints = backgroundpoints(nodata(background, d -> d < 4), 2sum(presencelayer))

In [ ]:
heatmap(L[1])
scatter!(presencelayer)
scatter!(bgpoints)
current_figure()

In [ ]:
sdm = SDM(PCATransform, DecisionTree, L, presencelayer, bgpoints)

In [ ]:
folds = kfold(sdm);
cv = crossvalidate(sdm, folds; threshold = true);
mean(mcc.(cv.validation))

In [ ]:
cv = crossvalidate(sdm, folds; threshold = true);
mean(mcc.(cv.validation))

In [ ]:
variables(sdm)

In [ ]:
train!(sdm)

In [ ]:
predict(sdm, L; threshold=true) |> heatmap

In [ ]:
ensemble = Bagging(sdm, 50)
bagfeatures!(ensemble)

In [ ]:
train!(ensemble)

In [ ]:
predict(ensemble, L; threshold=false) |> (s) -> heatmap(s, colormap=:thermal)
lines!(place[1].geometry, color=:black)
scatter!(presencelayer)
current_figure()